In [61]:
import numpy as np

def soft_thresholding(a, b):
    return np.sign(a) * np.maximum(np.abs(a) - b, 0)

def one_hot_encode(y, num_classes=None):
    y = np.array(y)
    if num_classes is None:
        num_classes = np.max(y) + 1
    
    one_hot = np.zeros((len(y), num_classes))
    one_hot[np.arange(len(y)), y] = 1
    return one_hot


class logisitic_regression():
    def __init__(self):
        pass
    
    def set_std_mean(self,X, epsilon = 1e-8):
        self.mean = np.mean(X, axis = 0)
        std = np.std(X,axis = 0)
        self.std = np.where(std > 0, std, epsilon)

    def standarize(self, X):
        return (X-self.mean)/self.std
        

    def fit(self, X, y, a, lambd, times = 10, extinction = 0.1):
        n, p = X.shape
        X, y = np.array(X), np.array(y)
        self.set_std_mean(X)
        X = self.standarize(X)
        y = one_hot_encode(y)
        g = y.shape[1]
        self.B = np.zeros((p,g))
        for i in range(times):
            #lambd *= extinction
            for k in range(g):
                for j in range(p):
                    xj = X[:,j]
                    #X = (X.T[self.B]).T jest tam opcja by olewać obliczenia dla zerowych bet to zrobić mniej więcej tak albo trzymać listę indeksów i na samych nich to robić
                    #X = X[:,non_zero_beta]
                    sum = (xj@(y[:,k])) - xj@X@(self.B[:,k]) + self.B[j,k]
                    self.B[j,k]= soft_thresholding(sum/n,lambd*a)/(1+lambd*(1-a))

    def predict_proba(self, X):
        X = np.array(X)
        X = self.standarize(X)
        X = np.exp(X@self.B)
        return X / X.sum(axis=1, keepdims=True)
        

In [30]:
from ucimlrepo import fetch_ucirepo
import numpy as np

rice_cammeo_and_osmancik = fetch_ucirepo(id=545) 
X_1 = np.array(rice_cammeo_and_osmancik.data.features) 
y_1 = (np.array(rice_cammeo_and_osmancik.data.targets).T[0]== "Cammeo").astype(int)

In [70]:
y_1

array([1, 1, 1, ..., 0, 0, 0])

In [89]:
lr_test = logisitic_regression()
lr_test.fit(X_1,y_1,1 ,0.1)

In [90]:
lr_test.B

array([[-0.03315889,  0.03315889],
       [-0.06549224,  0.06549224],
       [-0.02104434,  0.02104434],
       [-0.        ,  0.        ],
       [-0.01468034,  0.01468034],
       [-0.0674756 ,  0.0674756 ],
       [ 0.        , -0.        ]])

In [91]:
pred = lr_test.predict_proba(X_1).round()[:,1]

In [92]:
np.mean(y_1==pred)

0.9044619422572179